In [ ]:
import requests
import csv
import pandas as pd
import numpy as np
import geopandas
from pyproj import Proj, transform

In [ ]:
# SAM = geopandas.read_file('Live_Street_Address_Management_SAM_Addresses/Live_Street_Address_Management_SAM_Addresses.shp')
# SAM.crs

In [ ]:
df = pd.DataFrame()
df_all = pd.DataFrame()

In [ ]:
SAM = pd.read_csv('Live_Street_Address_Management_SAM_Addresses.csv')

inProj = Proj(init='epsg:2249')
outProj = Proj(init='epsg:4326')

In [ ]:
# Bing API
BingMapsAPIKey = "Ag9OGZ8tuL2y_uhT0cKaTg-tuS4wocFWrEmWaIWCcfcMoLC5_2Jma00ZhRVoT0XK"

In [ ]:
for idx in SAM.index[350000:398461]: 
    x = SAM['X'][idx]
    y = SAM['Y'][idx]
    longitude,latitude = transform(inProj,outProj,x,y)
    record = idx

    search_url = "http://dev.virtualearth.net/REST/v1/locationrecog/" + str(latitude) + "," + str(longitude) +\
                    "?key=" + BingMapsAPIKey + "&output=json"
    response = requests.get(search_url)
    response.raise_for_status()
    search_results = response.json()

    for i in range(len(search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'])):
        try:
            lat = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessAddress']['latitude']
            lon = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessAddress']['longitude']
        except:
            continue
        
        try:
            addressLine = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessAddress']['addressLine']
        except:
            addressLine = np.nan
            
        try:
            locality = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessAddress']['locality']
        except:
            locality = np.nan
            
        try:
            postalCode = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessAddress']['postalCode']
        except:
            postalCode = np.nan
            
        try:
            formattedAddress = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessAddress']['formattedAddress']
        except:
            formattedAddress = np.nan           
        
        try:
            entityName = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessInfo']['entityName']
        except:
            entityName = np.nan
            
        try:
            entityType = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessInfo']['type']
        except:
            entityType = np.nan
            
        try:
            otherTypes = search_results['resourceSets'][0]['resources'][0]['businessesAtLocation'][i]['businessInfo']['otherTypes']
        except:
            otherTypes = np.nan
        
        df = df.append({'LAT': lat, 
                        'LON': lon,
                        'ADDRESS_LINE': addressLine,
                        'LOCALITY': locality,
                        'ZIP_CODE': postalCode,
                        'FULL_ADDRESS': formattedAddress,
                        'BUSINESS_NAME': entityName,
                        'BUSINESS_TYPE': entityType,
                        'BUSINESS_OTHER_TYPES': otherTypes
                         }, 
                       ignore_index=True)

In [ ]:
df_all = df_all.append(df)
df_all = df_all.drop_duplicates(subset=['LAT','LON','FULL_ADDRESS','BUSINESS_NAME'], keep='first')
df_all = df_all.reset_index(drop=True)
df_all

In [ ]:
sequence = ['LAT','LON','ADDRESS_LINE','LOCALITY','ZIP_CODE','FULL_ADDRESS','BUSINESS_NAME','BUSINESS_TYPE','BUSINESS_OTHER_TYPES']
df_all = df_all.reindex(columns=sequence)

In [ ]:
# Yelp API
api_key = "hC_sbOQsbpD1Rm10RW-SU96tPGKryfjOznw0-mjbd6Lh2FYTR-KveTG5I1wia2vKwT7E3NDc4Bdh3r7IXetVhTz-FmjB4dkzr4OkOMRdgpGWlbv-YBn-ZkIpLJj7XnYx"
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(api_key)}

In [ ]:
df = pd.DataFrame()
for idx in coord.index[7621:7625]: 
    lat = coord['LAT'][idx]
    lon = coord['LON'][idx]
    record = idx

    url_params = {'latitude': lat,
                  'longitude': lon,
                  'radius': 50}

    response = requests.get(url, headers=headers, params=url_params)
    search_results = response.json()
    
    for i in range(len(search_results['businesses'])):
        try:
            latitude = search_results['businesses'][i]['coordinates']['latitude']
            longitude = search_results['businesses'][i]['coordinates']['longitude']
            businessID = search_results['businesses'][i]['id']
            name = search_results['businesses'][i]['name']
        except:
            continue
        
        try:
            is_closed = search_results['businesses'][i]['is_closed']
        except:
            is_closed = np.nan
            
        try:
            rating = search_results['businesses'][i]['rating']
        except:
            rating = np.nan
            
        try:
            phone = search_results['businesses'][i]['coordinates']['display_phone']
        except:
            phone = np.nan
        
        categories = []
        for j in range(len(search_results['businesses'][i]['categories'])):
            categories.append(search_results['businesses'][i]['categories'][j]['title'])
        
        if len(search_results['businesses'][i]['location']['display_address']) > 1:
            address1 = search_results['businesses'][i]['location']['display_address'][0]
            address2 = search_results['businesses'][i]['location']['display_address'][1]
            street_address = address1
            locality = address2
        else:
            address = search_results['businesses'][i]['location']['display_address'][0]
            locality = address
            
        df = df.append({'id': businessID,
                        'latitude': latitude, 
                        'longitude': longitude,
                        'street_address': street_address,
                        'locality': locality,
                        'business_name': name,
                        'categories': categories,
                        'is_closed': is_closed,
                        'phone': phone,
                        'rating': rating}, 
                       ignore_index=True)

In [ ]:
# OpenStreetMaps API
df = pd.DataFrame()
for idx in coord.index[6587:7625]: 
    lat = coord['LAT'][idx]
    lon = coord['LON'][idx]
    record = idx
    
    search_url = "https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat=" + str(lat) + "&lon=" + str(lon)
    response = requests.get(search_url)
    response.raise_for_status()
    search_results = response.json()
    
    for i in range(len(search_results)):
        try:
            osm_id = search_results['osm_id']
            latitude = search_results['lat']
            longitude = search_results['lon']
        except:
            continue
            
        try:
            name = search_results['address']['amenity']
        except:
            name = ''
            
        try:
            road = search_results['address']['road']
        except:
            road = ''
            
        try:
            neighbourhood = search_results['address']['neighbourhood']
        except:
            neighbourhood = ''
            
        try:
            suburb = search_results['address']['suburb']
        except:
            suburb = ''
            
        try:
            city = search_results['address']['city']
        except:
            city = ''
            
        try:
            county = search_results['address']['county']
        except:
            county = ''
            
        try:
            state = search_results['address']['state']
        except:
            state = ''
            
        try:
            postcode = search_results['address']['postcode']
        except:
            postcode = ''
    
        address = road + ', ' + neighbourhood + ', ' + suburb + ', ' + city + ', ' + \
                  county + ', ' + state + ', ' + postcode
        
    df = df.append({'osm_id': osm_id,
                    'latitude': latitude, 
                    'longitude': longitude,
                    'business_name': name,
                    'road': road,
                    'neighbourhood': neighbourhood,
                    'suburb': suburb,
                    'city': city,
                    'county': county,
                    'state': state,
                    'postcode': postcode,
                    'full_address': address},
                    ignore_index=True)

In [ ]:
df_all = df_all.append(df)
print(len(df_all))
df_all = df_all.drop_duplicates(subset=['business_name', 'full_address'], keep='first')
df_all = df_all.reset_index(drop=True)

In [ ]:
sequence = ['osm_id','latitude','longitude','business_name','road','neighbourhood','suburb','city','county',
            'state','postcode','full_address']
df_all = df_all.reindex(columns=sequence)
df_all

In [ ]:
df_all.to_excel('OSM.xlsx')

In [ ]:
f = pd.read_excel('merged_update.xlsx')
f

In [ ]:
InfoUSA_Only = f[(f['In InfoUSA list (Y/N)'] == 'Y') &
                 (f['In MSD list (Y/N)'] == 'N') &
                 (f['In Google list (Y/N)'] == 'N') &
                 (f['In Bing list (Y/N)'] == 'N')]

len(InfoUSA_Only)

In [ ]:
MG_Only = f[(f['In Yelp list (Y/N)'] == 'N') &
            (f['In Google list (Y/N)'] == 'N') &
            (f['In Bing list (Y/N)'] == 'N') &
            (f['In Yellowpages list (Y/N)'] == 'N') &
            (f['In Manual Google (Y/N)'] == 'Y')]
len(MG_Only)

In [ ]:
MSD_Only = f[(f['In MSD list (Y/N)'] == 'Y') &
              (f['In Google list (Y/N)'] == 'N') &
              (f['In Bing list (Y/N)'] == 'N') &
              (f['In Yelp list (Y/N)'] == 'N')]

len(MSD_Only)

In [ ]:
Yelp = f[(f['In Yelp list (Y/N)'] == 'Y')]
len(Yelp)

In [ ]:
Yelp_Google = f[(f['In Yelp list (Y/N)'] == 'Y') &
                (f['In Google list (Y/N)'] == 'Y')]
len(Yelp_Google)

In [ ]:
Yelp_MG = f[(f['In Manual Google (Y/N)'] == 'Y') &
            (f['In Yelp list (Y/N)'] == 'Y')]
len(Yelp_MG)

In [ ]:
Yelp_Only = f[(f['In MSD list (Y/N)'] == 'N') &
              (f['In Google list (Y/N)'] == 'N') &
              (f['In Bing list (Y/N)'] == 'N') &
              (f['In Yelp list (Y/N)'] == 'Y')]
len(Yelp_Only)

In [ ]:
InfoUSA = f[(f['In InfoUSA list (Y/N)'] == 'Y')]
len(InfoUSA)

In [ ]:
InfoUSA_Google = f[(f['In InfoUSA list (Y/N)'] == 'Y') &
                (f['In Google list (Y/N)'] == 'Y')]
len(InfoUSA_Google)

In [ ]:
InfoUSA_Bing = f[(f['In InfoUSA list (Y/N)'] == 'Y') &
                 (f['In Bing list (Y/N)'] == 'Y')]
len(InfoUSA_Bing)

In [ ]:
InfoUSA_YP = f[(f['In Yellowpages list (Y/N)'] == 'Y') &
                (f['In InfoUSA list (Y/N)'] == 'Y')]
len(InfoUSA_YP)

In [ ]:
SafeGraph = f[(f['In SafeGraph list (Y/N)'] == 'Y')]
len(SafeGraph)

In [ ]:
SafeGraph_Google = f[(f['In SafeGraph list (Y/N)'] == 'Y') &
                     (f['In Google list (Y/N)'] == 'Y')]
len(SafeGraph_Google)

In [ ]:
SafeGraph_Bing = f[(f['In SafeGraph list (Y/N)'] == 'Y') &
                   (f['In Bing list (Y/N)'] == 'Y')]
len(SafeGraph_Bing)

In [ ]:
SafeGraph_MG = f[(f['In SafeGraph list (Y/N)'] == 'Y') &
                 (f['In Manual Google (Y/N)'] == 'Y')]
len(SafeGraph_MG)

In [ ]:
SafeGraph_Unique = f[(f['In SafeGraph list (Y/N)'] == 'Y') &
                     (f['In Google list (Y/N)'] == 'N') &
                     (f['In Bing list (Y/N)'] == 'N') &
                     (f['In Yelp list (Y/N)'] == 'N') &
                     (f['In Yellowpages list (Y/N)'] == 'N') &
                     (f['In InfoUSA list (Y/N)'] == 'N') &
                     (f['In MSD list (Y/N)'] == 'N')]
len(SafeGraph_Unique)

In [ ]:
newf = f[f['In Manual Google (Y/N)'] == 'Y']

except_Google = newf[(newf['In Bing list (Y/N)'] == 'Y') |
                     (newf['In Yelp list (Y/N)'] == 'Y') |
                     (newf['In Yellowpages list (Y/N)'] == 'Y') |
                     (newf['In SafeGraph list (Y/N)'] == 'Y')]

len(except_Google)